# Exercice guidé : Télécharger des images depuis un manifeste IIIF

Dans cet exercice, nous allons apprendre à télécharger des images à partir d'un manifeste IIIF en utilisant Python. Nous allons parcourir les étapes suivantes :

1. **Ouvrir le manifeste**
2. **Parcourir le manifeste pour trouver les URLs des images**
3. **Utiliser la bibliothèque `requests` pour télécharger les images**
   - Afficher les codes de statut HTTP (200, 404, 429, etc.)
   - Récupérer les données
4. **Construire le chemin d'enregistrement des images avec `os` et les enregistrer**
5. **Utiliser la bibliothèque `PIL` pour enregistrer les caractéristiques de l'image dans un fichier `.txt` du même nom que l'image**
6. **Automatiser le processus avec une fonction pour récupérer toutes les URLs du manifeste**

---

## Prérequis

Avant de commencer, assurez-vous d'avoir les bibliothèques Python suivantes installées :

- `requests`
- `Pillow` (PIL)

Si vous travaillez sur colab activez la commande suivante dans une cellule python :
```bash
!pip install requests pillow
```
---

## Étape 1 : Ouvrir le manifeste

Un manifeste IIIF est un fichier JSON qui décrit une ressource numérique complexe, telle qu'une collection d'images. Pour cet exercice, nous utiliserons un manifeste d'exemple.

In [ ]:
import json
import requests

# URL du manifeste IIIF
manifest_url = 'https://www.e-codices.unifr.ch/metadata/iiif/bbb-0318/manifest.json'

# Télécharger le manifeste
response = requests.get(manifest_url)

# Vérifier que la requête s'est bien passée
if response.status_code == 200:
    manifest = response.json()
    print("Manifeste chargé avec succès.")
else:
    print(f"Erreur lors du chargement du manifeste : {response.status_code}")

### Explications

- **Importation des modules** : Nous importons les modules `json` pour manipuler le JSON et `requests` pour effectuer des requêtes HTTP.
- **Téléchargement du manifeste** : Nous utilisons `requests.get()` pour envoyer une requête GET à l'URL du manifeste.
- **Vérification du statut HTTP** : Nous vérifions si le `status_code` est 200, ce qui indique un succès.
- **Chargement du manifeste** : Si la requête a réussi, nous chargeons le contenu JSON du manifeste en utilisant `response.json()`.

---

## Étape 2 : Parcourir le manifeste pour trouver les URLs des images

Les images sont généralement situées dans les `canvases` du manifeste. Chaque canvas représente une image individuelle.

In [ ]:
# Extraire les canvases du manifeste
canvases = manifest['sequences'][0]['canvases']

# Créer une liste pour stocker les informations sur les images
images_info = []

# Parcourir les canvases pour extraire les informations
for idx, canvas in enumerate(canvases):
    canvas_num = idx
    image_label = canvas['label'] # Récupère l'index (position) du canvas dans le manifeste
    image_url = canvas['images'][0]['resource']['@id']
    image_format = canvas['images'][0]['resource']['@id'].split('/')[-1]
    image_width_declared = canvas['images'][0]['resource']['width']
    image_height_declared = canvas['images'][0]['resource']['height']
    
    images_info.append({
        'canvasNum': canvas_num,
        'imageLabel': image_label,
        'urlImage': image_url,
        'imageFormat' : image_format,
        'imageWidthAsDeclared': image_width_declared,
        'imageHeightAsDeclared': image_height_declared
    })

print(f"Trouvé {len(images_info)} images dans le manifeste.")

### Explications

- **Accès aux canvases** : Nous accédons à la liste des `canvases` du manifeste.
- **Utilisation de `enumerate()`** : L'utilisation de `enumerate()` dans cette boucle a pour but de parcourir les canvas du manifeste tout en obtenant leur index (position) en plus de leur valeur (l'élément lui-même). C'est particulièrement utile pour contrer les problème de nommage que vous pouvez rencontrer dans certains manifestes où tous les médias on le même `label`, le plus souvent `NP`.

- **Parcours des canvases** : Pour chaque canvas, nous extrayons :
  - `canvasNum` : Le numéro du canvas (son index).
  - `imageLabel` : Label de l'image, avec une valeur par défaut si absent.
  - `urlImage` : URL de l'image.
  - `imageFormat` : Le format de l'image (jpg, png, tiff, ...)
  - `imageWidthAsDeclared` et `imageHeightAsDeclared` : Dimensions déclarées de l'image.
- **Stockage des informations** : Nous ajoutons ces informations à la liste `images_info`.



---

## Étape 3 : Utiliser la bibliothèque `requests` pour télécharger les images

Nous allons maintenant utiliser `requests` pour télécharger les images à partir des URLs que nous avons extraites. Nous afficherons également les codes de statut HTTP.

In [ ]:
# Importer les modules nécessaires
import os

# Créer un dossier pour stocker les images téléchargées
folder_path = 'downloaded_images'
if not os.path.exists(folder_path):
    os.makedirs(folder_path)

# Parcourir la liste des images et les télécharger
for idx, image in enumerate(images_info):
    url = image['urlImage']
    image_filename = f"image_{idx+1}.jpg"
    image_path = os.path.join(folder_path, image_filename)
    
    try:
        response = requests.get(url)
        html_code = response.status_code
        print(f"Téléchargement de {image_filename} - Code HTTP : {html_code}")
        
        if response.status_code == 200:
            # Enregistrer l'image sur le disque
            with open(image_path, 'wb') as file:
                file.write(response.content)
            image['imageFileName'] = image_filename
            image['folderPath'] = folder_path
            image['htmlCode'] = html_code
        else:
            print(f"Erreur lors du téléchargement de l'image : {html_code}")
            image['htmlCode'] = html_code
    except Exception as e:
        print(f"Exception lors du téléchargement de l'image : {e}")
        image['htmlCode'] = 'Exception'

### Explications

- **Création du dossier** : Nous utilisons `os.makedirs()` pour créer un dossier `downloaded_images` si nécessaire.
- **Téléchargement des images** :
  - Nous parcourons chaque image dans `images_info`.
  - Nous construisons le nom du fichier image.
  - Nous utilisons `requests.get()` pour télécharger l'image.
  - Nous affichons le code HTTP pour chaque requête.
  - Si le téléchargement est réussi (`status_code` 200), nous enregistrons l'image et mettons à jour les informations dans `images_info`.
  - Nous gérons les exceptions pour capturer les erreurs éventuelles.

---

## Étape 4 : Construire le chemin d'enregistrement avec `os` et enregistrer l'image

Nous avons déjà utilisé `os` pour créer le dossier et construire le chemin du fichier. Nous allons continuer à utiliser `os` pour assurer la portabilité du code.

### Code

*Cette étape est intégrée dans le code de l'étape précédente.*

### Explications

- **Construction du chemin du fichier** : Nous utilisons `os.path.join()` pour créer un chemin compatible avec le système d'exploitation.
- **Vérification de l'existence du dossier** : `os.path.exists()` vérifie si le dossier existe déjà.
- **Création du dossier** : `os.makedirs()` crée le dossier spécifié.

---

## Étape 5 : Utiliser `PIL` pour enregistrer les caractéristiques de l'image

Nous allons maintenant utiliser `PIL` pour ouvrir l'image téléchargée et obtenir ses dimensions réelles. Nous enregistrerons les caractéristiques de l'image dans un fichier `.txt` du même nom que l'image.

In [ ]:
from PIL import Image
import os

# Parcourir les images pour récupérer les caractéristiques et les enregistrer
for image in images_info:
    image_filename = image.get('imageFileName')
    
    # Vérifier si l'image est valide (nom de fichier et code HTML = 200)
    if image_filename and image.get('htmlCode') == 200:
        image_path = os.path.join(image['folderPath'], image_filename)
        txt_filename = os.path.splitext(image_filename)[0] + '.txt'
        txt_path = os.path.join(image['folderPath'], txt_filename)
        
        # Ouvrir l'image avec PIL et récupérer les dimensions téléchargées (size retourne un tuple (width, height) )
        with Image.open(image_path) as img:
            image_width_downloaded, image_height_downloaded = img.size
            image['imageWidthAsDownloaded'] = image_width_downloaded
            image['imageHeightAsDownloaded'] = image_height_downloaded
        
        # Liste des clés à écrire dans le fichier .txt
        data_to_write = [
            'imageFileName', 'canvasId', 'urlImage', 'folderPath', 'imageLabel',
            'imageWidthAsDeclared', 'imageHeightAsDeclared', 'htmlCode',
            'imageWidthAsDownloaded', 'imageHeightAsDownloaded'
        ]
        
        # Enregistrer les caractéristiques dans le fichier .txt
        with open(txt_path, 'w', encoding='utf-8') as txt_file:
            for key in data_to_write:
                value = image.get(key)
                # Si la valeur est nulle (None ou chaîne vide), remplacer par "Non défini"
                if not value:
                    value = 'Non défini'
                txt_file.write(f"{key}: {value}\n")
        
        print(f"Caractéristiques enregistrées dans {txt_filename}")
    else:
        print(f"Image non téléchargée ou erreur lors du téléchargement pour {image.get('imageLabel', 'Inconnu')}")


### Explications

- **Ouverture de l'image avec PIL** : Nous utilisons `Image.open()` pour ouvrir l'image téléchargée.
- **Récupération des dimensions** : `img.size` renvoie les dimensions réelles de l'image sous la forme d'un tuple largeur, hauteur.
- **Création du fichier `.txt`** :
  - Nous changeons l'extension du nom de l'image en `.txt` en utilisant `os.path.splitext()`.
  - Nous écrivons les caractéristiques de l'image dans le fichier texte.
- **Gestion des erreurs** : 
  - Si l'image n'a pas été téléchargée avec succès, nous affichons un message.
  - Lors de l'écriture, si une clé est absente, une valeur par défaut sera retournée ('Non défini'). Cela évite les erreurs si certaines informations manquent.

- **Attention** : Le `get()` auquel nous faisons référence pour `image.get('imageFileName')` est une méthode des dictionnaires Python, utilisée pour récupérer la valeur associée à une clé spécifique dans un dictionnaire. Si la clé n'existe pas, `get()` retourne `None` (ou une valeur par défaut si spécifiée), ce qui évite de lever une exception (`KeyError`) comme avec l'accès direct par [].
---

## Étape 6 : Automatiser le processus avec une fonction

Pour rendre le processus réutilisable, vous allez encapsuler le code dans une fonction qui prend en entrée l'URL du manifeste et le dossier chemin vers le dossier de téléchargement.

In [ ]:
# Votre code ici

### Explications

- **Définition de la fonction** : `download_images_from_manifest()` prend en entrée l'URL du manifeste et le nom du dossier de destination.
- **Réutilisation du code** : L'objectif est d'encapsuler le code des étapes précédentes dans la fonction.
- **Appel de la fonction** : N'oubliez pas d'appeler la fonction!

---

## Conclusion

Dans cet exercice, nous avons appris à :

- **Ouvrir et analyser un manifeste IIIF** pour extraire les URLs des images.
- **Utiliser la bibliothèque `requests`** pour télécharger les images et gérer les codes de statut HTTP.
- **Utiliser la bibliothèque `os`** pour construire les chemins d'enregistrement et gérer les fichiers.
- **Utiliser la bibliothèque `PIL`** pour obtenir les caractéristiques des images téléchargées.
- **Automatiser le processus avec une fonction** pour le rendre réutilisable.

Ces compétences sont essentielles pour manipuler des ressources numériques dans le domaine de l'histoire numérique et pour gérer efficacement des collections d'images.

---

**Remarque** : Assurez-vous d'avoir une connexion Internet active lors de l'exécution de ce notebook, car il nécessite de télécharger des données en ligne.

---

# Exécution du notebook

Vous pouvez maintenant exécuter ce notebook cellule par cellule pour tester le processus complet de téléchargement d'images depuis un manifeste IIIF.

**Bon courage !**